In [23]:
import numpy as np
import pandas as pd
import pickle

import keras


# Load the model

In [24]:
loaded_model = keras.saving.load_model("model.keras")

# load the scaler and encoder
with open("scaler.pkl", "rb") as f: # feature scaling
    scaler = pickle.load(f)

with open("onehot_encoder_geo.pkl", "rb") as f:
    encoder_geo = pickle.load(f)

with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

# Example input data

In [25]:
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 40,
    "Tenure": 3,
    "Balance": 60000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}
# Convert the input data to a dataframe
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


# Preprocess the input data

In [26]:
geo_encoded = encoder_geo.transform(input_df[["Geography"]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df


/home/arman/miniconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


# Encode categorical columns

In [27]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


# Concatenate the encoded columns

In [28]:
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df = input_df.drop(columns=["Geography"])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


# Scale the input data

In [29]:
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

# Make a prediction

In [30]:
model_prediction = loaded_model.predict(input_scaled)
model_prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


array([[0.01121858]], dtype=float32)

In [32]:
if model_prediction[0][0] > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")


The customer is likely to stay with the bank.
